In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import sys
sys.path.append('/content/drive/My Drive/coarsen')
sys.path.append('/content/drive/My Drive/coarsen/data')

sys.path.append('/content/drive/My Drive/Coarsen Experiments on High Resolution Bunny')
import utils as ut

%load_ext autoreload
%autoreload 2
%aimport coarsening

import inspect
import os
import joblib
import tensorflow as tf
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle
import csv

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio
import process_data
import sklearn.metrics
import sklearn.neighbors
import scipy.sparse
import scipy.sparse.linalg
import scipy.spatial.distance
import skimage.measure
from sklearn.preprocessing import normalize
np.seterr(divide='ignore', invalid='ignore')

import matplotlib.cm
import random


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
#####################################
### UTILITY FUNCTIONS             ###
#####################################
def laplacian(W, normalized=True):
    """
    Return the Laplacian of the weigth matrix.
    W: input adjacency or weight matrix.
    
    """

    # Degree matrix.
    d = W.sum(axis=0)

    # Laplacian matrix.
    if not normalized:
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        L = D - W
    else:
        d += np.spacing(np.array(0, W.dtype))
        d = 1 / np.sqrt(d)
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        I = scipy.sparse.identity(d.size, dtype=W.dtype)
        L = I - D * W * D

    # assert np.abs(L - L.T).mean() < 1e-9
    assert type(L) is scipy.sparse.csr.csr_matrix
    return L
 


def coarsen(A, levels, normalized):
    graphs, parents = coarsening.metis(A, levels) #Coarsen a graph multiple times using Graclus variation of the METIS algorithm. 
                                                  #Basically, we randomly sort the nodes, we iterate on them and we decided to group each node
                                                  #with the neighbor having highest w_ij * 1/(\sum_k w_ik) + w_ij * 1/(\sum_k w_kj) 
                                                  #i.e. highest sum of probabilities to randomly walk from i to j and from j to i.
                                                  #We thus favour strong connections (i.e. the ones with high weight wrt all the others for both nodes) 
                                                  #in the choice of the neighbor of each node.
                    
                                                  #Construction is done a priori, so we have one graph for all the samples!
                    
                                                  #graphs = list of spare adjacency matrices (it contains in position 
                                                  #          0 the original graph)
                                                  #parents = list of numpy arrays (every array in position i contains 
                                                  #           the mapping from graph i to graph i+1, i.e. the idx of
                                                  #           node i in the coarsed graph -> that is, the idx of its cluster) 
    perms = coarsening.compute_perm(parents) #Return a list of indices to reorder the adjacency and data matrices so
                                             #that two consecutive nodes correspond to neighbors that should be collapsed
                                             #to produce the coarsed version of the graph.
                                             #Fake nodes are appended for each node which is not grouped with anybody else
    laplacians = []
    for i,A in enumerate(graphs):
        M, M = A.shape

        # We remove self-connections created by metis.
        A = A.tocoo()
        A.setdiag(0)

        if i < levels: #if we have to pool the graph 
            A = coarsening.perm_adjacency(A, perms[i]) #matrix A is here extended with the fakes nodes
                                                       #in order to do an efficient pooling operation
                                                       #in tensorflow as it was a 1D pooling

        A = A.tocsr()
        A.eliminate_zeros()
        Mnew, Mnew = A.shape
        print('Layer {0}: M_{0} = |V| = {1} nodes ({2} added), |E| = {3} edges'.format(i, Mnew, Mnew-M, A.nnz//2))

        L = laplacian(A, normalized)
        laplacians.append(L)

    return laplacians, parents, perms[0] if len(perms) > 0 else None



def poly_operator(L, r, coefficients):
    AA = sp.csr_matrix(L.shape)
    AA.setdiag(1.0)
    res = AA*coefficients[0]
    for k in range(r):
        AA = AA.dot(L)
        res = res + coefficients[k+1] * AA
    return res

  
# Sampling operator
def construct_S_for_L(perm, m, n, times): # m is the dimension of the coarsened graph, n is the original dimension, times is time you coarsen graph.
    S = sp.lil_matrix((m,n))
    for i in range(m):
      count = 0
      for k in range(times):
        if perm[times*i + k] <n:
          count = count + 1
      for k in range(times):
        if perm[times*i + k] <n:
          S[i,perm[times*i+k]] = 1.0/np.sqrt(count)
    S = S.tocsr()
    return S
               
def convert_map(perm):
    new_perm = np.arange(len(perm))
    new_perm = new_perm[perm]
    new_perm = np.argsort(new_perm)
    return new_perm

  
  
def convert_to_rgb(eigenvector):
    """ Input must be normalized eigenvector """
    
    cmap = matplotlib.cm.get_cmap('coolwarm')
    rgba = cmap(eigenvector) 
    rgba *= 255
    rgba = rgba.astype(int)   
    return rgba
  
def sigmoid(chosen_eigen, k):
   return 1/(1+np.exp(-1.0*k*chosen_eigen)) 
  
def customized_normalize(chosen_eigen):
    """ Normalize eigenvector before shifting and than dividing by max entry"""

    min_value = chosen_eigen.min(axis=0)
    chosen_eigen_shifted = chosen_eigen - min_value

    max_value = chosen_eigen_shifted.max(axis=0)
    chosen_eigen_shifted_normalized = chosen_eigen_shifted / max_value
    return chosen_eigen_shifted_normalized

"""
functions related to reading and writinging .off and .coff files
"""
def read_coff(file):
    if 'COFF' != file.readline().strip():
        raise('Not a valid COFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts_colors = [[float(s) for s in file.readline().strip().split(' ')[:-1]] for i in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts_colors, faces

def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces
file = open("/content/drive/My Drive/Coarsen Experiments on High Resolution Bunny/high_resolution_bunny.off", "r")
verts, faces = read_off(file)
coords = np.array(verts)
faces = np.array(faces)
file.close()

def write_coff(x_shift, y_shift, z_shift, signal, name, scale):
  """signal: original signal on bunny manifold
     name: name of your coff file to be saved
     scale: larger -> brighter
  """
  #processing signal before visualization
  signal = signal.real
  signal_normalized = customized_normalize(signal)
  colors = convert_to_rgb(signal_normalized)
  outfile = open("/content/drive/My Drive/Coarsen Experiments on High Resolution Bunny/high_resolution_plots/"+name+".off","w+")
  
  #don't change this part
  outfile.write("COFF\n")
  outfile.write("{0} {1} 0\n".format(len(coords), len(faces)))

  for i in range(len(coords)):
    outfile.write(str((coords[i,0] + x_shift)*scale) + " " + str((coords[i,1] + y_shift)*scale) + " " + str((coords[i,2] + z_shift)*scale) + " " + str(colors[i,0]) + " " + str(colors[i,1]) + " " + str(colors[i,2]) + " " + str(colors[i,3]) + " \n")
  
  for i in range(len(faces)):
    outfile.write("3" + " " + str(faces[i,0]) + " " + str(faces[i,1]) + " " + str(faces[i,2]) + "\n")
  outfile.close()

##**PART ONE: FINE TUNE PARAMETERS**
---


1.   construct signal 
2.   define filters 
3.   change brightness

In [0]:
#########################################################################
### Construct Signal  manully  with Ron's method                      ###
#########################################################################
distance0 = coords[:,0] 
distance0 = distance0 - np.min(distance0)


distance1 = coords[:,1] 
distance1 = distance1 - np.min(distance1)


distance2 = coords[:,2] 
distance2 = distance2 - np.min(distance2)

diam = np.max((np.max((np.max(distance0), np.max(distance1))), np.max(distance2)))
distance0 = distance0/diam
distance1 = distance1/diam
distance2 = distance2/diam

pi = 3.1415926
constructed_signal = distance0 * 0.0
coeffs = np.random.uniform(0,1,(10,10,10))

for i in range(10):
    for j in range(10):
        for k in range(10):
            constructed_signal = constructed_signal + np.exp(-2j*pi*coeffs[i,j,k]) * np.exp(-2j*pi*(i+1)*distance0) * np.exp(-2j*pi*(j+1)*distance1) * np.exp(-2j*pi*(k+1)*distance2)
constructed_signal = constructed_signal #Complex number
constructed_signal = constructed_signal.real
constructed_signal = 1.0/(1.0+np.exp(-1.0*constructed_signal))

In [0]:

r = 4 # the order of polinomial
"""
Middle pass: 2*x*(2-x)^3
"""
poly_coefficients_middle = 2.0*np.array([0.0, 8.0, -12.0, 6.0, -1.0])



In [0]:
shift = 6.0
scale = 1.5

##**PART TWO: WRITING COFF FILE**
*Do not change this part*
---


1.   original signal and interpolated signal.
2.   original signal after laplacian and interpolated coarsened signal after laplacian.
3.   original signal after filters and interpolated coarsened signal after filters.





In [0]:
#####################################
### IMPORT BUNNY ADJACENCY MATRIX ###
#####################################
A = sp.load_npz('/content/drive/My Drive/Coarsen Experiments on High Resolution Bunny/High_Resolution_Bunny_Adj.npz')
L = laplacian(A, True) # Normalized Laplacian

In [40]:
########################
### GRAPH COARSENING ###
########################
np.random.seed(0)

dim_graph = A.shape[0]

nonormal_laplacians, parents, perm = coarsen(A, 3, True)

new_perm = convert_map(perm)

S_L1 = construct_S_for_L(perm, nonormal_laplacians[1].shape[0], dim_graph, 2)
S_L2 = construct_S_for_L(perm, nonormal_laplacians[2].shape[0], dim_graph, 4)
S_L3 = construct_S_for_L(perm, nonormal_laplacians[3].shape[0], dim_graph, 8)

L_coarse1 = (S_L1.dot(L)).dot(S_L1.transpose())
L_coarse2 = (S_L2.dot(L)).dot(S_L2.transpose())
L_coarse3 = (S_L3.dot(L)).dot(S_L3.transpose())

Layer 0: M_0 = |V| = 42120 nodes (7303 added), |E| = 104445 edges
Layer 1: M_1 = |V| = 21060 nodes (2418 added), |E| = 55920 edges
Layer 2: M_2 = |V| = 10530 nodes (648 added), |E| = 29640 edges
Layer 3: M_3 = |V| = 5265 nodes (0 added), |E| = 15788 edges


In [0]:
#####################################
### COARSENING CONSTRUCTED SIGNAL ###
#####################################

signal_coarsen1 = S_L1.dot(constructed_signal)
signal_coarsen2 = S_L2.dot(constructed_signal)
signal_coarsen3 = S_L3.dot(constructed_signal)

In [0]:
##############################################################
### CREATE COLORED ORIGINAL AND INTERPOLATED BUNNY.OFF     ###
##############################################################
interpolated_signal_coarsen1 = (S_L1.transpose()).dot(signal_coarsen1)
interpolated_signal_coarsen2 = (S_L2.transpose()).dot(signal_coarsen2)
interpolated_signal_coarsen3 = (S_L3.transpose()).dot(signal_coarsen3)
write_coff(-shift, shift, 0, constructed_signal, 'constructed_signal', scale)
write_coff(-shift, 0, 0, interpolated_signal_coarsen1, 'interpolated_signal_coarsen1', scale)
write_coff(-shift, -shift, 0, interpolated_signal_coarsen2, 'interpolated_signal_coarsen2', scale)
write_coff(-shift, -2*shift, 0, interpolated_signal_coarsen3, 'interpolated_signal_coarsen3', scale)

In [0]:
######################################################
### APPLYING LAPLACIAN ON FINE AND COARSE MANIFOLD ###
######################################################
signal_after_laplacian = L.dot(constructed_signal)
signal_coarsen_after_laplacian1 = L_coarse1.dot(signal_coarsen1)
signal_coarsen_after_laplacian2 = L_coarse2.dot(signal_coarsen2)
signal_coarsen_after_laplacian3 = L_coarse3.dot(signal_coarsen3)
interpolated_signal_coarsen_after_laplacian1 = (S_L1.transpose()).dot(signal_coarsen_after_laplacian1)
interpolated_signal_coarsen_after_laplacian2 = (S_L2.transpose()).dot(signal_coarsen_after_laplacian2)
interpolated_signal_coarsen_after_laplacian3 = (S_L3.transpose()).dot(signal_coarsen_after_laplacian3)

In [0]:
######################################################
### CREATE COLORED AFTER LAPLACIAN BUNNY .OFF      ###
######################################################
write_coff(shift,shift,0,signal_after_laplacian, 'signal_after_laplacian', scale)
write_coff(shift,0,0,interpolated_signal_coarsen_after_laplacian1, 'interpolated_signal_coarsen_after_laplacian1', scale)
write_coff(shift,-shift,0,interpolated_signal_coarsen_after_laplacian2, 'interpolated_signal_coarsen_after_laplacian2', scale)
write_coff(shift,-2*shift,0,interpolated_signal_coarsen_after_laplacian3, 'interpolated_signal_coarsen_after_laplacian3', scale)

In [45]:
######################################
### DEFINE MIDDLE PASS FILTERS     ###
######################################
poly_operator_middle = poly_operator(L, r, poly_coefficients_middle)
poly_operator_coarse_middle1 = poly_operator(L_coarse1, r, poly_coefficients_middle)
poly_operator_coarse_middle2 = poly_operator(L_coarse2, r, poly_coefficients_middle)
poly_operator_coarse_middle3 = poly_operator(L_coarse3, r, poly_coefficients_middle)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [0]:
###################################################
### APPLYING FILTER ON FINE AND COARSE MANIFOLD ###
###################################################

signal_middle_filter = poly_operator_middle.dot(constructed_signal)
signal_coarsen_then_middle_filter1 = poly_operator_coarse_middle1.dot(signal_coarsen1)
signal_coarsen_then_middle_filter2 = poly_operator_coarse_middle2.dot(signal_coarsen2)
signal_coarsen_then_middle_filter3 = poly_operator_coarse_middle3.dot(signal_coarsen3)
interpolated_signal_coarsen_then_middle_filter1 = (S_L1.transpose()).dot(signal_coarsen_then_middle_filter1)
interpolated_signal_coarsen_then_middle_filter2 = (S_L2.transpose()).dot(signal_coarsen_then_middle_filter2)
interpolated_signal_coarsen_then_middle_filter3 = (S_L3.transpose()).dot(signal_coarsen_then_middle_filter3)

In [0]:
#coarsen_coords = S_for_features.dot(coords)
#from numpy import savetxt
#savetxt('/content/drive/My Drive/Coarsen Experiments on Bunny/coarsen_bunny_coordinates.csv', coarsen_coords, delimiter=' ')

In [0]:
######################################################
### CREATE COLORED AFTER FILTERS BUNNY .OFF        ###
######################################################
write_coff(0,shift,0,signal_middle_filter, 'signal_middle_filter', scale)
write_coff(0,0,0,interpolated_signal_coarsen_then_middle_filter1, 'interpolated_signal_coarsen_then_middle_filter1', scale)
write_coff(0,-shift,0,interpolated_signal_coarsen_then_middle_filter2, 'interpolated_signal_coarsen_then_middle_filter2', scale)
write_coff(0,-2*shift,0,interpolated_signal_coarsen_then_middle_filter3, 'interpolated_signal_coarsen_then_middle_filter3', scale)